In [1]:
import pathlib
import itertools
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops
import shapely

import pyproj

import requests
import networkx as nx



%matplotlib inline

In [2]:
data_dir = pathlib.Path('~/data/drought').expanduser()


In [3]:
ivs_path = data_dir / 'ivs' / 'ivs-geocoded.pickle'
graph_path = data_dir / 'network' / 'network_digital_twin_v0.3.pickle'

In [4]:
with ivs_path.open('rb') as f:
    ivs_gdf = pickle.load(f)
    ivs_gdf['geometry_herkomst'] = ivs_gdf['geometry'] 
with graph_path.open('rb') as f:
    graph = pickle.load(f)

In [5]:
# Remove open sea connections
graph.remove_edges_from([
    ('8867399', '8860964'),
    ('8860673', '8868194'),
    ('8862471', '8867736'),
    ('8862449', '8868149'), 
    ('8866969', '8860852'), # Rotterdam
    ('8863489', '8864829'),  # Scheveningen
    ('8860693', '8862971'), # IJmuiden
    ('8867031', '8860908'), # Den Helder
    ('8867286', '8867031'), # Den Helder
    ('8861237', '8866818'), # Vlieland
    ('8867280', '8867108'), # Terschelling
    ('8861800', '8862121'), # Ameland
    ('30986639', '8867432'), # Schiermonnikoog
    ('8867027', '8860544'), # Borkum
    ('30985121', '30986625'), #
    ('8860557', '8867857'),
])

In [6]:
graph.add_edges_from([
    ('L4583745_A', 'FN6'),
    ('FN264', '35228581'),
    ('L1278654_A', 'FN72'),
    ('L1278651_B', 'FN72'),
    ('L1361959_A', 'FN120'),
    ('35228529', 'FN120'),
    ('L1361139_A', 'FN647'),
    ('L1360434_B', 'FN647'),
    ('FN279', 'FN245'),
    ('FN302', 'FN68'),
    ('L1290993_B', 'FN219')
    
],
    **{'distance_m': 0}
)

In [7]:
routes_df = (
    ivs_gdf
    .dropna(subset=['n', 'n_bestemming'])
    .drop_duplicates(['n', 'n_bestemming'])
    [['n', 'n_bestemming']]
).reset_index(drop=True)

In [8]:
def find_route(row):
    """compute the route and compute the simple geometry"""
    has_fis_route = True
    try:
        route = nx.shortest_path(graph, source=row['n'], target=row['n_bestemming'], weight='distance_m')
    except nx.NetworkXNoPath:
        route = None
        has_fis_route = False
    
    if not route or len(route) == 1:
        a = shapely.geometry.shape(graph.nodes[row['n']]['geometry'])
        b = shapely.geometry.shape(graph.nodes[row['n_bestemming']]['geometry'])
        geometry = shapely.geometry.LineString([a, b])
        has_fis_route = False
    else:
        points = []
        for n in route:
            point = shapely.geometry.shape(graph.nodes[n]['geometry'])
            points.append(point)
        geometry = shapely.geometry.LineString(points)    
    
    row['geometry'] = geometry
    row['route'] = route
    row['has_fis_route'] = has_fis_route
    
    return row


routes_gdf = gpd.GeoDataFrame(routes_df.apply(find_route, axis=1))


In [9]:
routes_gdf.drop(columns=['route']).to_file(data_dir / 'network' / 'routes.geojson')

In [10]:
merged_gdf = ivs_gdf.merge(
    routes_gdf, 
    left_on=['n', 'n_bestemming'], 
    right_on=['n', 'n_bestemming'], 
    how='left', 
    suffixes=['', '_fis']
)


In [11]:
merged_gdf['has_fis_route'] = merged_gdf['has_fis_route'].fillna(0).astype('bool')

In [12]:
row = merged_gdf.loc[116367]


In [13]:
geod = pyproj.Geod(ellps='WGS84')
def great_circle_geometry(row):
    a = row['geometry_herkomst']
    b = row['geometry_bestemming']
    if a is None or b is None:
        return None
    intermediate = geod.inv_intermediate(a.x, a.y, b.x, b.y, npts=20)
    geometry = shapely.geometry.LineString(np.c_[intermediate.lons, intermediate.lats])
    return geometry

merged_gdf['geometry_great_circle'] = merged_gdf.apply(great_circle_geometry, axis=1)

In [14]:
merged_gdf['geometry'] = merged_gdf['geometry_fis'].where(merged_gdf['has_fis_route'], merged_gdf['geometry_great_circle'])

In [15]:
ivs_routed = merged_gdf[['geometry', 'jaar', 'week', 'reizen', 'beladen_lvm', 'lading', 'has_fis_route']]
ivs_routed = gpd.GeoDataFrame(ivs_routed)
ivs_routed.to_file(data_dir / 'ivs' / 'ivs_routed.geojson')



In [23]:
list(graph.nodes)